# Module 6 - MD Simulations of Simple Fluids, Lennard-Jones Potential, and Periodic Boundary Conditions

# Goals

- Learn the basics of the atomic simulation environment (ASE) python library
- Learn about periodic boundary conditions
- Run an MD simulation of simple solids and liquids
- Learn about non-bonded interaction potentials such as Lennard-Jones
- Learn aboout the Maxwell-Boltzmann distribution used to set the initial velocities

## Setting up a Crystal with the Atomic Simulation Environment (ASE)

ASE is "*a set of tools and Python modules for setting up, manipulating, running, visualizing and analyzing atomistic simulations.*" You can learn more about it on their website at https://wiki.fysik.dtu.dk/ase/ .

Let's start by building a simple system made of a bunch of argon (Ar) atoms in a face-centered cubic lattice. We will make the crystal by stacking 2x2x2 Argon unit cells in 3D. The lattice constant of Argon is 5.25 Angstroms. To do this, we will create an ASE `atoms` object using the `FaceCenteredCubic` method that is part of the `ase.latticle.cubic`. The created `atoms` object contains information about the type and number of atoms in your system, the positions of every atom, system dimensions, etc. We will use the ASE `units` method to add the right units to our quantities to avoid problems later on.

In [1]:
# Start by importing some specific ASE methods
from ase.lattice.cubic import FaceCenteredCubic
from ase import units

# Set up the crystal, which will be stored in an object called 'Ar_atoms'
# Our system is made of 2x2x2 unit cells with a total of 32 atoms (4 atoms per unit cell).
Ar_atoms = FaceCenteredCubic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                             symbol='Ar',
                             latticeconstant=5.25*units.Ang,
                             size=(2, 2, 2),
                             pbc=True)

We will use the built-in ASE `view` interface, part of the `ase.visualize` class to generate a 3D representation of our system. We will specify that `view` use the `nglview` (part of the NGL suite, https://nglviewer.org/) widget instead of the default `ase.gui` interface that requires an external viewer.

In [2]:
# Use the ASE viewer interface together with nglview to visualize the atoms
# Use the left mouse button to rotate the model
# Use scrolling to zoom in and out of the system
# The yellow box delimits the extent of the symulated system

from ase.visualize import view
view(atoms=Ar_atoms, viewer='ngl')

## Periodic Boundary Conditions
Although we are simulating a finite system (32 atoms total = 8 cells x 4 atoms/cell), we are going to model it as an infinitely periodic array using periodic boundary conditions (PBCs). To do this, our simulated system 'lives' inside a simulation cell or 'box' which is described by a set of 3 dimensions and 3 angles in the most general triclinic cell:

<center><img src="https://upload.wikimedia.org/wikipedia/commons/1/17/Triclinic.svg" width=200px></center>

For our purposes, we will deal with simple rectangular boxes where all three angles of the simulation cell are equal to 90 degrees, altough each dimension of the box can be different (i.e., the simulation cell does not need to be cubic). Using PBCs then assumes that our simulation cell is infinitely stacked along every dimension:

<center><img src="https://upload.wikimedia.org/wikipedia/commons/2/2e/Minimum_Image_Convention.png" width=400px></center>

PBCs allow us to avoid having artificial edges where the particles/molecules are exposed to vacuum and are not interacting with any other molecules. When working with PBCs, we also impose the **minimum image convention** - particles only interact with particles that are in the neighboring image that is closest to them. This means that while particles have an infinite number of neighbors, they only interact with other particles if the distance along every dimension is less than half of the box size in that given dimension.

<center><img src="https://computecanada.github.io/molmodsim-md-theory-lesson-novice/fig/periodic_boundary-4.svg" width=500px></center>



## The Lennard-Jones Potential

Now that we have a starting structure for our system, how are the particles going to interact with each other? One commonly used interatomic potential is known as the Lennard-Jones or 12-6 potential. This potential is named after Sir John Edward Lennard Jones who studied the attractive forces, also known as van der Waals or London dispersion forces, in noble gases. The Lennard-Jones potential is composed of an attractive component that decays proportionally to $1/r^6$, where $r$ is the distance between particles, and a repulsive term that rapidly grows proportionally to $1/r^{12}$ to keep particles from getting too close to each other (i.e., from Pauli repulsion forces). The $1/r^6$ attractive term is obtained analitically by quantifying the forces due to the spontaneous fluctuations in the electron clouds of atoms, while the $1/r^{12}$ relationship is chosen ad-hoc for numerical efficiency reasons as one can compute it as the square of the attractive term, $1/r^{12}=1/(r^6)^2$. The full potential is written as

\begin{equation}
    V_{\mathrm{LJ}}(r) = 4 \epsilon \left[ \left( \frac{\sigma}{r} \right) ^{12} - \left( \frac{\sigma}{r} \right) ^6 \right],
\end{equation}

where $\sigma$ is the effective particle radius and $\epsilon$ is the depth of the energy well as shown below:
<center><img src="https://computecanada.github.io/molmodsim-md-theory-lesson-novice/fig/lj.svg" width=500px></center>

Some sources use the alternative notation of 

\begin{equation}
    V_{\mathrm{LJ}}(r) = \frac{C_{12}}{r^{12}} - \frac{C_6}{r^6},
\end{equation}

where $C_{12}=4 \epsilon \sigma^{12}$ and $C_{6}=4 \epsilon \sigma^{6}$.

The force due to the Lennard-Jones potential is given by the derivative with respect to the particle distance:

\begin{equation}
     F_{\mathrm{LJ}}(r) = -\frac{\partial V_{\mathrm{LJ}}(r)}{\partial r}  = 24 \epsilon \left[ 2\left( \frac{\sigma}{r} \right) ^{12} - \left( \frac{\sigma}{r} \right) ^6 \right].
\end{equation}

### Using the `LennardJones` calculator in ASE

ASE has a built-in `calculator` that automatically computes interatomic forces for a given system using the Lennard-Jones potential. An ASE `calculator` is a method that takes as input an `atoms` object and computes quantities such as the forces and total energy (among many others) for a particular configuration. ASE `calculators` may execute internal built-in methods or external programs that can simulate systems using a variety of classical or *ab initio* methods. We will import the `LennardJones` calculator and add it to our `atoms` object. We also need to tell our calculator the specific values of the potential for Ar including $\sigma=3.4~\unicode[serif]{xC5}$, $\epsilon=0.9889744$ kJ/mol, and the cutoff radius, $r_c=15~\unicode[serif]{xC5}$. The cutoff radius is used to speed-up calculations by limiting the number of neighbors that particles interact with. A cutoff radius of $10-15~\unicode[serif]{xC5}$ is used for most Lennard-Jones potentials in MD simulations.  

In [3]:
from ase.calculators.lj import LennardJones as LJ

# Describe the interatomic interactions using a Lennard-Jones potential
Ar_atoms.calc = LJ(sigma=3.4*units.Ang, epsilon=0.9889744*units.kJ/units.mol, rc=15*units.Ang)

## Initial Velocities and the Maxwell-Boltzmann Distribution

When running a molecular dynamics simulation we need to know the initial particle velocities in addition to the initial positions in order to use the Verlet algorithm to propagate the equations of motion in time. While we could just set the initial velocities to zero, this is equivalent to setting the initial temperature of the system to zero. A better approach is to use the Maxwell-Boltzmann distribution, which states that the particle velocities in an ideal gas follow the relationship

\begin{equation}
    P(v)= \left[ \frac{m}{2\pi k_{\mathrm{B}}T} \right]^{\frac{3}{2}} 4 \pi v^2 \exp \left( -\frac{mv^2}{2k_{\mathrm{B}}T} \right),
\end{equation}

where $m$ is the particle mass, $v=|\vec{v}|$ is the velocity, $k_{\mathrm{B}}$ is Boltzmann's constant, and $T$ is the temperature. 

<center><img src="https://tikz.net/janosh/maxwell-boltzmann-dist.png" width=500px></center>

To use the Maxwell-Boltzmann distribution we randomly assign particle velocities such that they follow the $P(v)$ distribution above. We will use the `MaxwellBoltzmannDistribution` method from the `ase.md.velocitydistribution` class to set the initial velocities according to a temperature of 40 K - this will keep our Ar crystal in the solid phase.

In [4]:
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

# Set the initial particle velocities using a Maxwell-Boltzmann distribution for a given temperature
MaxwellBoltzmannDistribution(atoms=Ar_atoms, temperature_K=40)

## Running an MD Simulation Using the `VelocityVerlet` Method

We are now ready to run our MD simulation! We will first create a new instance called `md` using the `VelocityVerlet` method from the `ase.md.verlet` class. The `VelocityVerlet` method takes as input the `Ar_atoms` object we created earlier, the timestep (5 femtoseconds is adequate for our system), a trajectory file to store the positions and velocities of our system over time, and the interval to write to the trajectory file. Once the `md` instance is created, you can run it for a given number of steps (e.g., 100) by executing `md.run(100)`. Let's put it all together:

In [5]:
# Start by importing some specific ASE methods
from ase.lattice.cubic import FaceCenteredCubic
from ase import units
from ase.calculators.lj import LennardJones as LJ
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet


# Set up the crystal, which will be stored in an object called 'Ar_atoms'
# Our system is made of 2x2x2 unit cells with a total of 32 atoms (4 atoms per unit cell).
Ar_atoms = FaceCenteredCubic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                             symbol='Ar',
                             latticeconstant=5.25*units.Ang,
                             size=(2, 2, 2),
                             pbc=True)

# Describe the interatomic interactions using a Lennard-Jones potential
Ar_atoms.calc = LJ(sigma=3.4*units.Ang, epsilon=0.9889744*units.kJ/units.mol, rc=15*units.Ang)

# Set the initial particle velocities using a Maxwell-Boltzmann distribution for a given temperature
MaxwellBoltzmannDistribution(atoms=Ar_atoms, temperature_K=40)

# Setup an md instance based on our `Ar_atoms` object
md = VelocityVerlet(atoms=Ar_atoms, timestep=5*units.fs, trajectory='md.traj', loginterval=10)  # 5 fs time step.

# run the simulation for 100 steps
md.run(100)

True

We'll need to load the trajectory file as an ASE `Trajectory` object before we can visualize it.

In [6]:
from ase.visualize import view
from ase.io.trajectory import Trajectory

# Load the trajectory from file
traj = Trajectory('md.traj')
view(traj, viewer='ngl')

<div class="span alert alert-success">
<h2> Programming challenge </h2>
    
Modify the code above to increase the number of steps to 1000 and run the simulation again. This should take about 30 s. Visually inspect the trajectory by rotating the model and replaying the movie. What do you observe? **The system appears to be drifting!!** 
    
The reason the entire crystal appears to be drifting out of the simulation box is because the process of assigning initial velocities from the Maxwell-Boltzmann distribution can result in a net velocity of the center of mass, $\vec{v}_{\mathrm{COM}}= \frac{1}{M} \sum m_i \vec{v}_i \neq 0$. We can fix this in our script by calling the `ase.md.velocitydistribution.Stationary` method after the call to `MaxwellBoltzmannDistribution`.
    
Copy the working code above and paste it in the cell below. Modify it to import the `Stationary` method from `ase.md.velocitydistribution`. Call `Stationary(Ar_atoms)` after calling `MaxwellBoltzmannDistribution(atoms=Ar_atoms, temperature_K=40)`. Execute and visualize your code again. Do you still see a drift in the system?

In [ ]:
# Start by importing some specific ASE methods
from ase.lattice.cubic import FaceCenteredCubic
from ase import units
from ase.calculators.lj import LennardJones as LJ
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution,Stationary
from ase.md.verlet import VelocityVerlet


# Set up the crystal, which will be stored in an object called 'Ar_atoms'
# Our system is made of 2x2x2 unit cells with a total of 32 atoms (4 atoms per unit cell).
Ar_atoms = FaceCenteredCubic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                             symbol='Ar',
                             latticeconstant=5.25*units.Ang,
                             size=(2, 2, 2),
                             pbc=True)

# Describe the interatomic interactions using a Lennard-Jones potential
Ar_atoms.calc = LJ(sigma=3.4*units.Ang, epsilon=0.9889744*units.kJ/units.mol, rc=15*units.Ang)

# Set the initial particle velocities using a Maxwell-Boltzmann distribution for a given temperature
MaxwellBoltzmannDistribution(atoms=Ar_atoms, temperature_K=40)
Stationary(Ar_atoms)

# Setup an md instance based on our `Ar_atoms` object
md = VelocityVerlet(atoms=Ar_atoms, timestep=5*units.fs, trajectory='md.traj', loginterval=10)  # 5 fs time step.

# run the simulation for 1000 steps
md.run(1000)

In [ ]:
# Load the trajectory from file
traj = Trajectory('md.traj')
view(traj, viewer='ngl')

## Simulating Liquid Argon

We can readily simulate liquid Argon by increasing the initial temperature of the Maxwell-Boltzmann distribution as well as increasing the box size in order to accomodate for the lower density of the liquid.

<div class="span alert alert-success">
<h2> Programming challenge </h2>

Copy the working code above and paste it in the cell below. Modify the code so that the lattice constant is increased by 15 % and the initial temperature is 150 K. Execute your code for 1000 steps and visualize the trajectory.

In [ ]:
# Set up the crystal, which will be stored in an object called 'Ar_atoms'
# Our system is made of 2x2x2 unit cells with a total of 32 atoms (4 atoms per unit cell).
Ar_atoms = FaceCenteredCubic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                             symbol='Ar',
                             latticeconstant=1.15*5.25*units.Ang,
                             size=(2, 2, 2),
                             pbc=True)

# Describe the interatomic interactions using a Lennard-Jones potential
Ar_atoms.calc = LJ(sigma=3.4*units.Ang, epsilon=0.9889744*units.kJ/units.mol, rc=15*units.Ang)

# Set the initial particle velocities using a Maxwell-Boltzmann distribution for a given temperature
MaxwellBoltzmannDistribution(atoms=Ar_atoms, temperature_K=150)
Stationary(Ar_atoms)

# Setup an md instance based on our `Ar_atoms` object
md = VelocityVerlet(atoms=Ar_atoms, timestep=5*units.fs, trajectory='md.traj', loginterval=10)  # 5 fs time step.

# run the simulation for 1000 steps
md.run(1000)

In [ ]:
# Load the trajectory from file
traj = Trajectory('md.traj')
view(traj, viewer='ngl')